In [48]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [49]:
#loader =PyPDFLoader("data/W1L1_Ch1 Part1.pdf")
loader =PyPDFDirectoryLoader("data/")

In [50]:
pages = loader.load_and_split()

In [51]:
pages[-1]

Document(metadata={'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'creationdate': '2025-02-15T10:24:26-08:00', 'title': 'Figures-Chapter', 'author': 'Ian Sommerville', 'moddate': '2025-02-15T10:24:26-08:00', 'source': 'data\\W2L2.pdf', 'total_pages': 27, 'page': 26, 'page_label': '27'}, page_content='Key points\n\uf0b2Processes should include activities to cope with change. \nThis may involve a prototyping phase that helps avoid \npoor decisions on requirements and design. \n\uf0b2Processes may be structured for iterative development \nand delivery so that changes may be made without \ndisrupting the system as a whole.\n\uf0b2The Rational Unified Process is a modern generic \nprocess model that is organized into phases (inception, \nelaboration, construction and transition) but separates \nactivities (requirements, analysis and design, etc.) from \nthese phases.\n27Chapter 2 Software Processes')

In [52]:
for i in range(5,10):
    print(pages[i].page_content)
    print('\n\n')

Product specification
Generic products
▪ The specification of what the software should do is owned by the 
software developer and decisions on software change are made 
by the developer.
Customized products
▪ The specification of what the software should do is owned by the 
customer for the software and they make decisions on software 
changes that are required.
6



Frequently asked questions about software 
engineering
7
Question Answer
What is software? Computer programs and associated documentation. 
Software products may be developed for a particular 
customer or may be developed for a general market.
What are the attributes of good software? Good software should deliver the required functionality 
and performance to the user and should be 
maintainable, dependable and usable.
What is software engineering? Software engineering is an engineering discipline that is 
concerned with all aspects of software production.
What are the fundamental software 
engineering activities?
Softwa

In [53]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap= 20 )
documents = text_splitter.split_documents(pages)

In [54]:
print(len(pages))
print(len(documents))

104
108


In [55]:
import os 
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [56]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_type=OPENAI_API_KEY)

In [57]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

vector_store.save_local("faiss_index")

new_vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

In [58]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(documents))]
new_vector_store.add_documents(documents=documents, ids=uuids)

['05055bb8-5580-4814-b1de-b37f54e2acb7',
 'f08d6737-19c3-4705-b1fa-1bc1367ba758',
 '5f31c39b-0202-4f4d-a7c0-7c9b3e01518c',
 'bfe6ad8c-be09-4dd6-aec8-212e590b6e59',
 '8f7f6d27-edcd-489e-9272-eb59b72aaefd',
 '30b8269d-0d75-4b11-857d-3a497fc315df',
 '20bd1c8f-473f-4eac-9134-8129c9b713d5',
 'db97fe2a-5634-4521-9fbd-832066f5cec9',
 'f517038f-680b-4c0c-a7bd-d3bf84030560',
 '8861a67f-842c-48d9-808d-c4f88d6e35f2',
 '64ef81b7-c5a3-49f6-95ff-cb43e78b4c5f',
 '2cd8dcc0-5af7-4d81-9571-457f80b5fd15',
 'bc01eab5-bb75-4ca2-a30f-351e4fe04bf1',
 'ee99f260-57e6-4662-9c5e-4c9089866a74',
 '7586a239-b54f-447c-846c-39ec37772716',
 'db84642b-7768-436c-82ba-7af93eecfc96',
 '2800561e-6403-43e0-abc8-665689fe45a1',
 '95c0a3bf-ee7e-42e7-91b1-9e304730cb08',
 '0905ae62-6ab6-4116-acbf-7c4b9049a615',
 'a9cd1f32-1a08-41ca-8a98-f7e32777ee56',
 'ab6acbfc-5672-41c2-96c5-7415ae082930',
 '2b622369-2993-47ac-bbf0-a5336400e441',
 '2907ed14-e8f0-4295-8b9f-a61c5b65517f',
 'ed4c1e31-a063-411e-9a2e-65a0ac8a19e8',
 'a5e89fb2-cf0d-

In [59]:
from langchain_openai import ChatOpenAI
#llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)
llm = ChatOpenAI(api_key=OPENAI_API_KEY)

In [60]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [61]:
retriever = new_vector_store.as_retriever()

In [62]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

instruction_to_system = """
Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. DO NOT answer the question, just reformulate it if needed and otherwise return it as it is"""

question_maker_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", instruction_to_system),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

question_chain = question_maker_prompt| llm | StrOutputParser()

In [63]:
from langchain_core.messages import AIMessage, HumanMessage
question_chain.invoke({"question":"can you please expalin more?",
                       "chat_history":[HumanMessage(content="you explained that the moon is round")]})

'Can you provide more information about the shape of the moon?'

In [64]:
qa_system_prompt = """You are an intelligent and highly precise Software Engineering Course Assistant for COMP3401 at SQU. 
Your primary role is to analyze multiple text-based PDFs (including important tables and occasional images) and extract 
**only** the exact answers based on user queries.

### 🔹 Rules & Behavior:
1️⃣ **Strict Answer Extraction**:  
   - Respond **exclusively** using the content from the provided PDFs.  
   - **Do NOT generate** any information beyond what is explicitly stated in the documents.  

2️⃣ **Smart PDF Selection**:  
   - Analyze the question carefully.  
   - Choose the **most relevant PDF(s)** that contain the correct answer.  

3️⃣ **Accurate Data Handling**:  
   - Extract **tables accurately** and format them properly.  
   - Ensure important tabular data is **clear and well-structured** in your response.  

4️⃣ **Strict Relevance**:  
   - If the answer **cannot** be found in the PDFs, clearly state:  
     **"I cannot find this information in the provided documents."**  

5️⃣ **Citations & Transparency**:  
   - Always include the **exact source** of your response in this format:  
     **"Answer found in [Document Name], Page X."**  

6️⃣ **Handling Conflicting Information**:  
   - If multiple PDFs provide **different answers**, list **all conflicting responses**.  
   - Mention discrepancies **briefly** as a **note** for the user.  

7️⃣ **Summarized Responses**:  
   - If the answer is lengthy, provide a **clear and concise summary** while maintaining accuracy.  

8️⃣ **PDF Source Indication**:  
   - Always specify **which PDF(s) were used** to generate the response.  

---

### ❓ Types of User Questions:
1. **Normal Questions**:  
   - Users will ask typical questions looking for answers directly from the PDFs.  
   - Provide a **direct, concise** response from the retrieved PDF(s).

2. **Summarize the PDFs**:  
   - Users might ask you to **summarize** the content of the PDFs.  
   - In such cases, provide a **concise summary** covering the main points without losing the key information.

3. **Multiple-Choice Questions**:  
   - Users may present a multiple-choice question with available options.  
   - Choose the **correct answer** based on the information in the retrieved PDFs.  
   - Your response should include:  
     - The **chosen answer**  
     - A **justification** explaining why this is the correct choice based on the PDFs.  

4. **True/False Statements**:  
   - Users may provide a **statement** and ask you to verify if it is **true** or **false**.  
   - Use your **deep understanding** of the PDFs to evaluate the statement.  
   - If the statement is true, state:  
     - **"The statement is true."**  
   - If false, state:  
     - **"The statement is false."**  
     - Provide a brief **justification** for why it is false.

---

### 📝 **Example Response Format**:
#### **Normal Question Answer:**
- **Answer**:  
  "According to [Document Name], Page X: [Extracted Answer]."

#### **Multiple-Choice Question Answer:**
- **Answer**:  
  "**Choice A** is the correct answer."  
- **Justification**:  
  "This is because, according to [Document Name], Page X: [Extracted Information]."

#### **True/False Statement Answer:**
- **Answer**:  
  "**True**" or "**False**".  
- **Justification**:  
  "This is based on the information found in [Document Name], Page X: [Extracted Information]."

---

### 🔎 **Use the following retrieved context to answer the question:**  
{context}  
"""


qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{question}")
    ]
)

In [65]:
def contextualized_question(input: dict):
    if input.get("chat_history"):
        return question_chain
    else:
        return input["question"]

In [66]:
from langchain_core.runnables import RunnablePassthrough
retriever_chain = RunnablePassthrough.assign(
    context = contextualized_question | retriever 
)

In [67]:
retriever_chain.invoke({
    "chat_history":[HumanMessage(content="")],
    "question": "What is software ?"
})

{'chat_history': [HumanMessage(content='', additional_kwargs={}, response_metadata={})],
 'question': 'What is software ?',
 'context': [Document(id='8f7f6d27-edcd-489e-9272-eb59b72aaefd', metadata={'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'creationdate': '2025-02-15T10:12:53-08:00', 'title': 'Figures – Chapter 1', 'author': 'Ian Sommerville', 'moddate': '2025-02-15T10:12:53-08:00', 'source': 'data\\W1L1_Ch1 Part1.pdf', 'total_pages': 23, 'page': 4, 'page_label': '5'}, page_content='Software products\n\uf0b2Generic products\n▪ Stand-alone systems that are marketed and sold to any \ncustomer who wishes to buy them.\n▪ Examples – PC software such as graphics programs, project \nmanagement tools; CAD software; software for specific markets \nsuch as appointments systems for dentists.\n\uf0b2Customized products\n▪ Software that is commissioned by a specific customer to meet \ntheir own needs. \n▪ Examples – embedded cont

In [68]:
rag_chain = (
    retriever_chain
    | qa_prompt
    | llm
    | output_parser
)

In [69]:
question = "what are Waterfall model problems ?"

In [70]:
chat_history =[]
ai_msg = rag_chain.invoke({"question":question, "chat_history":chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])
ai_msg

"- **Waterfall model problems**:\n  - Inflexible partitioning of the project into distinct stages makes it difficult to respond to changing customer requirements.\n    - Therefore, this model is only appropriate when the requirements are well-understood and changes will be fairly limited during the design process.\n    - Few business systems have stable requirements.\n  - The waterfall model is mostly used for large systems engineering projects where a system is developed at several sites.\n    - In those circumstances, the plan-driven nature of the waterfall model helps coordinate the work.\n\n**Source**:  \nAnswer found in [Document 'W2L1.pdf'], Page 8."